# AbsBox Structured Finance Analysis Examples

This notebook demonstrates how to use the AbsBox integration to analyze structured finance products.

In [ ]:
import sys
import os
from pathlib import Path
from datetime import date, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

# Add the parent directory to the path so we can import from app
sys.path.append(str(Path.cwd().parent))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / ".env.test")

# Import AbsBox and related modules
from app.services.absbox_service import AbsBoxService
from app.models.structured_products import (
    StructuredDealRequest,
    LoanPoolConfig,
    LoanConfig,
    WaterfallConfig,
    AccountConfig,
    BondConfig,
    WaterfallAction,
    ScenarioConfig,
    DefaultCurveConfig,
    RateCurveConfig
)

## 1. Service Health Check

First, let's check that the AbsBox service is working correctly.

In [ ]:
# Initialize the AbsBox service
service = AbsBoxService()

# Check the service health
health_status = service.health_check()
print(json.dumps(health_status, indent=2))

## 2. Creating a Simple Mortgage Pool

Let's create a simple pool of mortgage loans.

In [ ]:
# Define start date (1 month ago)
start_date = date.today() - timedelta(days=30)

# Create a mortgage pool configuration
def create_mortgage_pool(num_loans=10, avg_balance=200000.0, avg_rate=0.05, term=360):
    """Create a pool of mortgage loans with some randomization"""
    np.random.seed(42)  # For reproducibility
    
    loans = []
    for i in range(num_loans):
        # Randomize loan parameters slightly
        balance = avg_balance * (0.8 + 0.4 * np.random.random())
        rate = avg_rate * (0.9 + 0.2 * np.random.random())
        
        loan = LoanConfig(
            balance=round(balance, 2),
            rate=round(rate, 4),
            term=term,
            start_date=start_date,
            rate_type="fixed",
            payment_frequency="Monthly"
        )
        loans.append(loan)
    
    pool_config = LoanPoolConfig(
        pool_name="Mortgage Pool Example",
        loans=loans
    )
    
    return pool_config

# Create a pool with 10 loans
mortgage_pool = create_mortgage_pool(num_loans=10)

# Display the loan details
loan_details = pd.DataFrame([
    {
        "loan_id": i,
        "balance": loan.balance,
        "rate": loan.rate,
        "term": loan.term,
        "payment_freq": loan.payment_frequency
    }
    for i, loan in enumerate(mortgage_pool.loans)
])

print(f"Created a mortgage pool with {len(mortgage_pool.loans)} loans")
loan_details

## 3. Creating a Simple Securitization Structure

Now, let's create a simple waterfall structure for our mortgage pool.

In [ ]:
# Calculate the total pool balance
total_pool_balance = sum(loan.balance for loan in mortgage_pool.loans)
print(f"Total pool balance: ${total_pool_balance:,.2f}")

# Create a waterfall structure
waterfall = WaterfallConfig(
    start_date=date.today(),
    accounts=[
        AccountConfig(name="Collection", initial_balance=0.0),
        AccountConfig(name="Reserve", initial_balance=total_pool_balance * 0.02)  # 2% reserve
    ],
    bonds=[
        # Senior tranche (80% of pool)
        BondConfig(name="Senior", balance=total_pool_balance * 0.8, rate=0.04),
        # Mezzanine tranche (15% of pool)
        BondConfig(name="Mezzanine", balance=total_pool_balance * 0.15, rate=0.055),
        # Junior/Equity tranche (5% of pool)
        BondConfig(name="Junior", balance=total_pool_balance * 0.05, rate=0.07)
    ],
    actions=[
        # Senior interest payments
        WaterfallAction(source="CollectedInterest", target="Senior", amount="Interest"),
        # Mezzanine interest payments
        WaterfallAction(source="CollectedInterest", target="Mezzanine", amount="Interest"),
        # Junior interest payments
        WaterfallAction(source="CollectedInterest", target="Junior", amount="Interest"),
        # Senior principal payments
        WaterfallAction(source="CollectedPrincipal", target="Senior", amount="OutstandingPrincipal"),
        # Mezzanine principal (paid after Senior)
        WaterfallAction(source="CollectedPrincipal", target="Mezzanine", amount="OutstandingPrincipal"),
        # Junior principal (paid last)
        WaterfallAction(source="CollectedPrincipal", target="Junior", amount="OutstandingPrincipal")
    ]
)

# Display bond structure
bond_structure = pd.DataFrame([
    {
        "tranche": bond.name,
        "balance": bond.balance,
        "rate": bond.rate,
        "percent_of_pool": round(bond.balance / total_pool_balance * 100, 2)
    }
    for bond in waterfall.bonds
])

print("Securitization Structure:")
bond_structure

## 4. Defining Analysis Scenarios

Let's define baseline and stress scenarios for our analysis.

In [ ]:
# Define scenarios
base_scenario = ScenarioConfig(
    name="Base Case",
    default_curve=DefaultCurveConfig(
        vector=[0.01, 0.015, 0.02, 0.015, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01]
    )
)

stress_scenario = ScenarioConfig(
    name="Stress Case",
    default_curve=DefaultCurveConfig(
        vector=[0.03, 0.045, 0.06, 0.055, 0.05, 0.04, 0.03, 0.025, 0.02, 0.015]
    )
)

recovery_scenario = ScenarioConfig(
    name="Recovery Case",
    default_curve=DefaultCurveConfig(
        vector=[0.02, 0.015, 0.01, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005, 0.005]
    )
)

# Plot default curves
plt.figure(figsize=(10, 6))
plt.plot(base_scenario.default_curve.vector, label="Base Case", marker='o')
plt.plot(stress_scenario.default_curve.vector, label="Stress Case", marker='s')
plt.plot(recovery_scenario.default_curve.vector, label="Recovery Case", marker='^')
plt.title("Default Rate Scenarios")
plt.xlabel("Period")
plt.ylabel("Default Rate")
plt.legend()
plt.grid(True)
plt.show()

## 5. Creating and Analyzing the Deal

Now let's put everything together and analyze the deal.

In [ ]:
# Create the deal request with the base scenario
deal_request = StructuredDealRequest(
    deal_name="Mortgage Securitization Example",
    pool=mortgage_pool,
    waterfall=waterfall,
    scenario=base_scenario
)

# Analyze the deal
print("Analyzing deal...")
result = service.analyze_deal(deal_request)

print(f"Analysis completed in {result.execution_time:.2f} seconds")
print(f"Status: {result.status}")

if result.status == "error":
    print(f"Error: {result.error}")
else:
    print(f"Generated {len(result.bond_cashflows)} cashflow periods")

## 6. Analyzing Bond Cashflows

Let's visualize the bond cashflows.

In [ ]:
# Convert to DataFrame
if result.status == "success":
    df_cf = pd.DataFrame(result.bond_cashflows)
    
    # Convert date columns
    if 'date' in df_cf.columns:
        df_cf['date'] = pd.to_datetime(df_cf['date'])
    
    # Plot total cashflows by tranche
    plt.figure(figsize=(12, 6))
    
    # Plot each tranche
    tranches = [bond.name for bond in waterfall.bonds]
    for tranche in tranches:
        if tranche in df_cf.columns:
            plt.plot(df_cf['date'], df_cf[tranche], label=tranche)
    
    plt.title("Bond Cashflows by Tranche")
    plt.xlabel("Date")
    plt.ylabel("Cashflow Amount")
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Show first few cashflow periods
    print("First 5 cashflow periods:")
    df_cf.head()

## 7. Running Scenario Analysis

Let's run multiple scenarios and compare the results.

In [ ]:
# Define the scenarios to run
scenarios = [base_scenario, stress_scenario, recovery_scenario]

# Run the scenarios
print("Running scenario analysis...")
scenario_results = service.run_scenario_analysis(deal_request, scenarios)

# Display results
results_summary = []
for result in scenario_results:
    scenario_data = {
        "scenario": result.scenario_name,
        "status": result.status,
        "execution_time": result.execution_time
    }
    
    # Add bond metrics if available
    for tranche in ["Senior", "Mezzanine", "Junior"]:
        if tranche in result.bond_metrics:
            metrics = result.bond_metrics[tranche]
            if "irr" in metrics:
                scenario_data[f"{tranche}_irr"] = metrics["irr"]
            if "loss" in metrics:
                scenario_data[f"{tranche}_loss"] = metrics["loss"]
    
    results_summary.append(scenario_data)

# Convert to DataFrame
df_results = pd.DataFrame(results_summary)
df_results

## 8. Visualizing Scenario Comparison

Let's create some visualizations to compare the scenarios.

In [ ]:
# Plot IRR comparison if available
if 'Senior_irr' in df_results.columns:
    # Create bar chart of IRRs by tranche and scenario
    tranches = ["Senior", "Mezzanine", "Junior"]
    irr_data = {}
    
    for tranche in tranches:
        irr_column = f"{tranche}_irr"
        if irr_column in df_results.columns:
            irr_data[tranche] = df_results[irr_column].values
    
    if irr_data:
        # Create DataFrame for plotting
        plot_data = pd.DataFrame(irr_data, index=df_results['scenario'])
        
        # Plot
        ax = plot_data.plot(kind='bar', figsize=(10, 6))
        plt.title("IRR by Tranche and Scenario")
        plt.xlabel("Scenario")
        plt.ylabel("IRR")
        plt.grid(axis='y')
        plt.xticks(rotation=0)
        
        # Add value labels
        for container in ax.containers:
            ax.bar_label(container, fmt='%.2f%%', padding=3)
            
        plt.tight_layout()
        plt.show()
        
# Plot loss comparison if available
if 'Senior_loss' in df_results.columns:
    # Create bar chart of losses by tranche and scenario
    tranches = ["Senior", "Mezzanine", "Junior"]
    loss_data = {}
    
    for tranche in tranches:
        loss_column = f"{tranche}_loss"
        if loss_column in df_results.columns:
            loss_data[tranche] = df_results[loss_column].values
    
    if loss_data:
        # Create DataFrame for plotting
        plot_data = pd.DataFrame(loss_data, index=df_results['scenario'])
        
        # Plot
        ax = plot_data.plot(kind='bar', figsize=(10, 6))
        plt.title("Loss by Tranche and Scenario")
        plt.xlabel("Scenario")
        plt.ylabel("Loss %")
        plt.grid(axis='y')
        plt.xticks(rotation=0)
        
        # Add value labels
        for container in ax.containers:
            ax.bar_label(container, fmt='%.2f%%', padding=3)
            
        plt.tight_layout()
        plt.show()

## 9. Summary and Key Metrics

Let's summarize the key metrics from our analysis.

In [ ]:
# Assuming we have pool_metrics in the result
if result.status == "success":
    print("Deal Summary:")
    print(f"Deal Name: {result.deal_name}")
    print(f"Execution Time: {result.execution_time:.2f} seconds")
    
    # Pool statistics
    if result.pool_statistics:
        print("\nPool Statistics:")
        for key, value in result.pool_statistics.items():
            print(f"  {key}: {value}")
    
    # Print bond metrics
    if 'bond_metrics' in result.metrics:
        print("\nBond Metrics:")
        for bond, metrics in result.metrics['bond_metrics'].items():
            print(f"  {bond}:")
            for metric, value in metrics.items():
                print(f"    {metric}: {value}")
                
    # Print pool metrics
    if 'pool_metrics' in result.metrics:
        print("\nPool Metrics:")
        for metric, value in result.metrics['pool_metrics'].items():
            print(f"  {metric}: {value}")